# Using native Python

## Default local client

When rgxlog is imported, a default client is created behind the scenes. This is the client that %%spanner uses.
However, one can also use his own client in the following manner:

In [ ]:
from rgxlog import Client


client = Client()
result = client.execute('''
    new uncle(str, str)
    uncle("bob", "greg")
    ?uncle(X,Y)
    ''')
print(result)
# Notice the default port is taken by the default client, hence the next port is used for the user's client

In [ ]:
client = None  # Notice the connection closes, due to GC
# client.disconnect()  # if you wish to disconnect explicitly you can do so as well

## Manually configured remote client

The following cell demonstrates connecting to a remote server.
* Start the remote server (run server.py)
* Note down the server's port and ip
* Use the following code snippet with your port and ip to connect

In [ ]:
from rgxlog import Client
# the server can also be remote

custom_client = Client(
    remote_ip='192.168.0.100',
    remote_port=32678
)

result = custom_client.execute('''
    new uncle(str, str)
    uncle("bob", "greg")
    ?uncle(X,Y)
    ''')
print(result)

custom_client.disconnect()

# Mixing magics and native python

In [ ]:
import rgxlog  # default client starts here, and captures 32768
from rgxlog import Client

another_client=Client()  # therefore this client will try the next in line, 32769

In [ ]:
%%spanner
# still using the default client

new uncle(str, str)
uncle("bob", "greg")
?uncle(X,Y)

In [ ]:
default_client = rgxlog.magic_client  # save the default for later (prevents GC)
rgxlog.magic_client = another_client

In [ ]:
%%spanner
# using 'another_client', not the default

new uncle(str, str)
uncle("bob", "greg")
?uncle(X,Y)

In [ ]:
another_client.disconnect()
rgxlog.magic_client = default_client

In [ ]:
%%spanner
# back to using the default client
new uncle(str, str)
uncle("bob", "greg")
?uncle(X,Y)

# Registering custom IE functions

If you wish to implement your own IE extraction you can check out python_regexes.py for a concise demo on how to do so. Once you make such a function you can register it as follows:

In [ ]:
rgxlog.magic_client.register('RGXString')

Using an IE function after registration is simple:

In [ ]:
%%spanner
report = "In 2019 we earned 2000 EUR"
annual_earning(Year,Amount) <- RGXString(report,"(\d\d\d\d).*?(\d+)")->(Year,Amount)
?annual_earning(X,Y)

## Default ie functions

If you wish to have certain IE functions registered by default, all you have to do is add them to "default_ie" list in server.py's main.